In [ ]:
import os
from dotenv import load_dotenv 
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage


os.environ["LANGSMITH_PROJECT"] = "AgenticAIworkspace"
load_dotenv()
groq_key = os.getenv("GROQ_API_KEY")

In [ ]:
messages = [AIMessage(content="How can i help today",name="LLMModel")]

messages.append(HumanMessage(content="my name is abdullah", name="abdullah"))
messages.append(AIMessage(content="Hello Abdullah", name="LLMModel"))
messages.append(HumanMessage(content="what is your model name", name="abdullah"))



In [ ]:
llm = ChatGroq(api_key=groq_key, model="llama-3.1-8b-instant")

In [ ]:
# create custom tool to connect toe LLM -> as a Brain 🧠
def add(a: int, b: int)-> int:
    """ Add a and b
    Args: 
        a (int) first int
        b (int) second int
    Return:
        int
    """
    return a + b

In [ ]:
## binding llm + tool
llm_with_tools=llm.bind_tools([add])

tool_call=llm_with_tools.invoke([HumanMessage(content="what is 2 plus 2", name="abdullah")])

In [ ]:
tool_call.tool_calls

In [ ]:
# using messages as state
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages: list[AnyMessage]


In [ ]:
from langgraph.graph.message import add_messages
from typing import Annotated
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
initial_messages = [AIMessage(content="How can i help today",name="LLMModel")]
initial_messages.append(HumanMessage(content="my name is abdullah", name="abdullah"))

initial_messages

In [ ]:
ai_messages = AIMessage(content="Hello Abdullah", name="LLMModel")
ai_messages

In [ ]:
add_messages(initial_messages, ai_messages)

In [ ]:
# chat bot node funcnatity
def llm_tool(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
from IPython.display import display,Image
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)
builder.add_node("llm_tool", llm_tool)
builder.add_edge(START, "llm_tool")
builder.add_edge("llm_tool", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
# invocation
messages = graph.invoke({"messages": "what is 2 plus 2"})

for message in messages:
    print(message)

In [ ]:
tools= [add]

In [ ]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition


builder = StateGraph(State)
builder.add_node("llm_tool", llm_tool)
builder.add_node("tools", ToolNode(tools))
# add edge
builder.add_edge(START, "llm_tool")
builder.add_conditional_edges("llm_tool", tools_condition)
builder.add_edge("tools", END)

graph_builder = builder.compile()

In [ ]:
display(Image(graph_builder.get_graph().draw_mermaid_png()))


In [ ]:
# invocation
from pprint import PrettyPrinter
messages = graph_builder.invoke({"messages": "what is 1 plus 2"})

for message in messages["messages"]:
    message.pretty_print()

In [ ]:
# invocation
from pprint import PrettyPrinter
messages = graph_builder.invoke({"messages": " what is machin learning?"})

for message in messages["messages"]:
    message.pretty_print()

    